# EDA

Let's have a first look at the data. 

The Variables in the files are:
* S1: heat power
* S2: flow rate
* S3: leader temperature (initial temperature)
* S4: return temperature

For the years 2020 and 2021 we also have the outside temperature at the location of the heat plant. 

We also have: 
* holydays in Bavaria (a lot!)
* school vacations in Bavaria

We will first focus only on the years 2020 and 2021, since here, we have all the data. For 2022, temperature is missing. 

## Read the data

In [1]:
import pandas as pd
import seaborn as sns

In [5]:
df_2020 = pd.read_csv("../raw_data/2020_heat.csv", delimiter=";", index_col=False)
df_2021 = pd.read_csv("../raw_data/2021_heat.csv", delimiter=";", index_col=False)

df = pd.concat([df_2020, df_2021])
df.rename({"S1": "heat_power", "S2": "flow_rate", "S3": "leater_temp", "S4": "return_temp"}, axis=1, inplace=True)
df

,Timestamp,heat_power,flow_rate,leater_temp,return_temp
0,2020-01-01T00:15:00.000000+01:00,713.000,17650.000,81.000,46.000
1,2020-01-01T00:30:00.000000+01:00,330.000,9600.000,81.000,51.000
2,2020-01-01T00:45:00.000000+01:00,705.000,16730.000,81.000,44.000
3,2020-01-01T01:00:00.000000+01:00,663.000,16890.000,81.000,47.000
4,2020-01-01T01:15:00.000000+01:00,412.000,10690.000,81.000,48.000
...,...,...,...,...,...
34939,2021-12-30T23:00:00.000000+01:00,385.571,17884.286,81.857,62.857
34940,2021-12-30T23:15:00.000000+01:00,436.500,20311.250,81.875,62.750
34941,2021-12-30T23:30:00.000000+01:00,320.143,16380.000,81.714,64.714
34942,2021-12-30T23:45:00.000000+01:00,302.250,17442.500,82.000,66.625
